In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [ ]:
# --- 1. El Bloque Residual (el corazón de ResNet) ---
def bloque_residual(x, filtros, kernel_size=3, stride=1):
    """
    Un bloque residual con "skip connection".
    """
    # Guarda la entrada original para el atajo
    x_atajo = x

    # Primer camino convolucional
    x = Conv2D(filtros, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Segundo camino convolucional (sin activación aún)
    x = Conv2D(filtros, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # skip connection
    # Si el stride es mayor a 1, la dimensión espacial (ancho/alto) cambia.
    # Necesitamos que la entrada del atajo tenga la misma dimensión que la salida del camino principal.
    # Para ello, usamos una convolución de 1x1 en el atajo para ajustar las dimensiones.
    if stride > 1:
        x_atajo = Conv2D(filtros, kernel_size=1, strides=stride, padding='same')(x_atajo)
        x_atajo = BatchNormalization()(x_atajo)

    # Sumamos la salida del camino principal con la del atajo
    x = Add()([x, x_atajo])
    # Aplicamos la activación DESPUÉS de la suma
    x = Activation('relu')(x)

    return x

In [ ]:
def crear_mini_resnet(input_shape=(32, 32, 3), num_clases=10):
    """
    Construye una Mini-ResNet apilando bloques residuales.
    """
    # Capa de entrada
    inputs = Input(shape=input_shape)

    # Capa convolucional inicial (preparación)
    x = Conv2D(32, kernel_size=3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Apilamos los bloques residuales
    x = bloque_residual(x, filtros=32)
    x = bloque_residual(x, filtros=32)

    # Reducimos el tamaño y aumentamos filtros
    x = bloque_residual(x, filtros=64, stride=2)
    x = bloque_residual(x, filtros=64)

    # Capas finales para la clasificación
    x = AveragePooling2D(pool_size=8)(x)
    x = Flatten()(x)
    outputs = Dense(num_clases, activation='softmax')(x)

    # Creamos el modelo final
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# --- 3. Creación y Resumen del Modelo ---
# Definimos la forma de entrada (ej: para imágenes CIFAR-10) y el número de clases
INPUT_SHAPE = (32, 32, 3)
NUM_CLASES = 10

modelo_resnet = crear_mini_resnet(input_shape=INPUT_SHAPE, num_clases=NUM_CLASES)

modelo_resnet.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
modelo_resnet.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │      9,248 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │      9,248 │ activation_1[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 32, 32,    │          0 │ add[0][0]         │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 32, 32,    │      9,248 │ activation_2[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │      9,248 │ activation_3[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 32, 32,    │          0 │ batch_normalizat

 Total params: 173,770 (678.79 KB)

 Trainable params: 172,810 (675.04 KB)

 Non-trainable params: 960 (3.75 KB)